# PyTorch Tutorial 08 - Logistic Regression

In [ ]:
# Steps:
# 1: Design our model (input_size, output_size, forward_pass)
# 2: Construct loss and optimizer
# 3: Training loop: we repeat in several epoch
# - forward pass: compute prediction
# - backward pass: calculate gradients
# - update our weights


In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [120]:
# 0) prepare dataset
# this is a binary classification problem
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape
print("samples:", n_samples)
print("features:", n_features)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

sc = StandardScaler()  # makes mean = 0, unit variance
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

# 1) model
# f = wx + b -> sigmoid function
class LogisticRegression(nn.Module):
    
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted
    

model = LogisticRegression(n_features)

# 2) loss & optimizer
# BCELoss -> Binary Cross Entropy Loss function
criterion = nn.BCELoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 3) training loop
n_epochs = 100

for epoch in range(n_epochs):
    # forward pass & loss calculation
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)

    # backward pass
    loss.backward()

    # updates
    optimizer.step()
    optimizer.zero_grad()

    if (epoch + 1)%10 == 0:
        print(f"epoch: {epoch+1}, loss: {loss.item():.4f}")

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_classes = y_predicted.round()
    acc = y_predicted_classes.eq(y_test).sum() / float(y_test.shape[0])
    print(f"accuracy: {acc:.4f}")

samples: 569
features: 30
epoch: 10, loss: 0.8043
epoch: 20, loss: 0.6044
epoch: 30, loss: 0.4901
epoch: 40, loss: 0.4186
epoch: 50, loss: 0.3699
epoch: 60, loss: 0.3346
epoch: 70, loss: 0.3077
epoch: 80, loss: 0.2865
epoch: 90, loss: 0.2693
epoch: 100, loss: 0.2549
accuracy: 0.9298
